# Introduction
The dataset for this competition (both train and test) was generated from a deep learning model trained on the [California housing dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html).
Feature distributions are close to, but not exactly the same, as the original.

This dataset was derived from the 1990 U.S. census, using one row per census block group.
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

Data ranges, and data types for each feature in the data set are as follows, their names are pretty self explanitory:
* `MedInc` - Median income in block group
* `HouseAge` - Median house age in block group
* `AveRooms` - Average number of rooms per household
* `AveBedrms` - Average number of bedrooms per household
* `Population` - Block group population
* `AveOccup` - Average number of household members
* `Latitude` - Block group latitude
* `Longitude` - Block group longitude

The evaluation metric is going to be the standard Root Mean Squared Error (RMSE)!

**If you found this notebook useful, please upvote!**  
**Thank you!**🙏

# Library Import
* Some library import and some configurations of seaborn.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import fetch_california_housing

import lightgbm as lgbm
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

sns.set_theme(style='darkgrid')

# Data

In [ ]:
DATAPATH = Path('../input/playground-series-s3e1')

train_set = pd.read_csv(DATAPATH/'train.csv')
test_set = pd.read_csv(DATAPATH/'test.csv')
sample_sub = pd.read_csv(DATAPATH/'sample_submission.csv')

In [ ]:
train_set.head()

# EDA

### Data Size
* The dataset is small. We can try out more advanced training and ensembling techniques!

In [ ]:
print('train_set shape: ', train_set.shape)
print('test_set shape: ', test_set.shape)

In [ ]:
train_set.describe()

## Missing Data
* There are no missing values!

In [ ]:
total = train_set.isnull().sum().sort_values(ascending=False)
percent = (train_set.isnull().sum()/train_set.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)

## Data visualisation

In [ ]:
features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
target = ['MedHouseVal']

### Data Distributions
* There isn't notable differences between train and test

In [ ]:
# Train set
fig, axes = plt.subplots(3,3, figsize=(20, 12))
for i, j in zip(features+target, axes.flatten()):
    sns.histplot(train_set[i], ax=j)
plt.show()

In [ ]:
# Test set
fig, axes = plt.subplots(4,2, figsize=(15, 12))
for i, j in zip(features, axes.flatten()):
    sns.histplot(test_set[i], ax=j)
fig.tight_layout()
plt.show()

### **Bivariate Distribution with Target**
* Greater MedInc the greater the MedHouseVal

In [ ]:
fig, axs = plt.subplots(2,4, figsize=(20, 8))
for i, j in enumerate(features):
    var = j
    plt.subplot(2,4,i+1)
    data = pd.concat([train_set['MedHouseVal'], train_set[var]], axis=1)
    plt.scatter(x=data[var], y=data['MedHouseVal'])
    plt.xlabel(j)
    plt.ylabel('MedHouseVal')
plt.show()

### Correlation matrix (Heatmap Style)
* Greater MedInc the greater the MedHouseVal, validated from correlations and from bivariate distribution!
* Correlations from train and original datasets are different.

In [ ]:
# train set
corrmat = train_set[features+target].corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, square=True, annot=True, fmt='.2f', cmap='seismic', vmin=-1, vmax=1)

### Longitude and Latitude
* By rotating the coordinates, they may provide more spatial information!

In [ ]:
plt.figure(figsize = (12, 8))

sns.scatterplot(data = train_set, x = "Longitude", y = "Latitude",
                size = "MedHouseVal", hue = "MedHouseVal",
                palette = "winter", alpha = 0.5)

plt.legend(title = "MedHouseVal", bbox_to_anchor = (1.05, 0.95), loc = 2)
plt.title("Median house value - spatial location")
plt.show()

In [ ]:
def rt_crds(df): 
    
    df['rot_15_x'] = (np.cos(np.radians(15)) * df['Longitude']) + \
                      (np.sin(np.radians(15)) * df['Latitude'])
    
    df['rot_15_y'] = (np.cos(np.radians(15)) * df['Latitude']) - \
                      (np.sin(np.radians(15)) * df['Longitude'])
    
    df['rot_30_x'] = (np.cos(np.radians(30)) * df['Longitude']) + \
                      (np.sin(np.radians(30)) * df['Latitude'])
    
    df['rot_30_y'] = (np.cos(np.radians(30)) * df['Latitude']) - \
                      (np.sin(np.radians(30)) * df['Longitude'])
    
    df['rot_45_x'] = (np.cos(np.radians(45)) * df['Longitude']) + \
                      (np.sin(np.radians(45)) * df['Latitude'])
    
    df['rot_45_y'] = (np.cos(np.radians(45)) * df['Latitude']) - \
                      (np.sin(np.radians(45)) * df['Longitude'])

    return df

train_set = rt_crds(train_set)
test_set = rt_crds(test_set)

### Rotating the coordinates!

In [ ]:
plt.figure(figsize = (12, 8))

sns.set_theme(style='darkgrid')
sns.scatterplot(data = train_set, x = "rot_45_x", y = "rot_45_y",
                size = "MedHouseVal", hue = "MedHouseVal",
                palette = "winter", alpha = 0.5)

plt.legend(title = "MedHouseVal", bbox_to_anchor = (1.05, 0.95), loc = 2)
plt.title("Median house value - spatial location")
plt.show()

# Train Model

In [ ]:
features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 
            'rot_15_x', 'rot_15_y', 'rot_30_x', 'rot_30_y', 'rot_45_x', 'rot_45_y']
target = ['MedHouseVal']

### XGBRegressor

In [ ]:
import warnings
warnings.filterwarnings('ignore')

kf = KFold(n_splits=10, random_state=42, shuffle=True)
clfs = []
err = []

for i, (train_index, val_index) in enumerate(kf.split(train_set)):
    X_train, X_val = train_set[features].loc[train_index], train_set[features].loc[val_index]
    y_train, y_val = train_set['MedHouseVal'][train_index], train_set['MedHouseVal'][val_index]
    
    clf = XGBRegressor(n_estimators=20000,
                       max_depth=9,
                       learning_rate=0.01,
                       colsample_bytree=0.66,
                       subsample=0.9,
                       min_child_weight=22,
                       reg_lambda=16,
                       tree_method='gpu_hist',
                       seed=42)
    
    clf.fit(X_train.values, y_train,
            early_stopping_rounds=100, 
            eval_set=[(X_val.values, y_val)], 
            verbose=1000)
    
    preds = clf.predict(X_val.values)
    
    rmse = mean_squared_error(y_val, preds, squared=False)
    err.append(rmse)
    clfs.append(clf)
    print(f'RMSE on fold {i}: {rmse}')
    print('-'*50)

print(f'Average RMSE (five fold): {sum(err)/10}')

### LGBMRegressor

In [ ]:
err = []

for i, (train_index, val_index) in enumerate(kf.split(train_set)):
    X_train, X_val = train_set[features].loc[train_index], train_set[features].loc[val_index]
    y_train, y_val = train_set['MedHouseVal'][train_index], train_set['MedHouseVal'][val_index]
    
    clf = lgbm.LGBMRegressor(learning_rate=0.01,
                             max_depth=9,
                             num_leaves=90,
                             colsample_bytree=0.8,
                             subsample=0.9,
                             subsample_freq=5,
                             min_child_samples=36,
                             reg_lambda=28,
                             n_estimators=20000,
                             metric='rmse',
                             random_state=42)
    
    clf.fit(X_train.values, y_train, eval_set=[(X_val.values, y_val)], callbacks=[lgbm.early_stopping(100, verbose=True)])
    preds = clf.predict(X_val.values)
    
    rmse = mean_squared_error(y_val, preds, squared=False)
    err.append(rmse)
    clfs.append(clf)
    print(f'RMSE on fold {i}: {rmse}')
    print('-'*50)

print(f'Average RMSE (five fold): {sum(err)/10}')

### CatBoostRegressor

In [ ]:
err = []

for i, (train_index, val_index) in enumerate(kf.split(train_set)):
    X_train, X_val = train_set[features].loc[train_index], train_set[features].loc[val_index]
    y_train, y_val = train_set['MedHouseVal'][train_index], train_set['MedHouseVal'][val_index]
    
    clf = CatBoostRegressor(iterations=20000,
                            depth=9,
                            learning_rate=0.01,
                            rsm=0.88,
                            subsample=0.795,
                            min_data_in_leaf=35,
                            l2_leaf_reg=8,
                            random_strength=0.63,
                            bootstrap_type='Bernoulli',
                            grow_policy='SymmetricTree',
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            task_type="CPU",
                            random_state=42)
    
    clf.fit(X_train.values, y_train, eval_set=(X_val.values, y_val), early_stopping_rounds=100, verbose=1000)
    preds = clf.predict(X_val.values)
    
    rmse = mean_squared_error(y_val, preds, squared=False)
    err.append(rmse)
    clfs.append(clf)
    print(f'RMSE on fold {i}: {rmse}')
    print('-'*50)

print(f'Average RMSE (five fold): {sum(err)/10}')

## Feature Importance

In [ ]:
imp = np.zeros(14)
for clf in clfs[:10]:
    imp+= clf.feature_importances_
    
print('----------------------------XGBoost----------------------------')
plt.barh([features[i] for i in np.argsort(imp/10)], sorted(imp/10))
plt.show()

In [ ]:
imp = np.zeros(14)
for clf in clfs[10:20]:
    imp+= clf.feature_importances_
    
print('----------------------------LGBM----------------------------')
plt.barh([features[i] for i in np.argsort(imp/10)], sorted(imp/10))
plt.show()

In [ ]:
imp = np.zeros(14)
for clf in clfs[20:]:
    imp+= clf.feature_importances_
    
print('----------------------------CatBoost----------------------------')
plt.barh([features[i] for i in np.argsort(imp/10)], sorted(imp/10))
plt.show()

# Making submission

In [ ]:
test_preds1 = []
test_preds2 = []
test_preds3 = []

for clf in clfs[:10]:
    preds = clf.predict(test_set[features].values)
    test_preds1.append(preds)
    
for clf in clfs[10:20]:
    preds = clf.predict(test_set[features].values)
    test_preds2.append(preds)
    
for clf in clfs[20:]:
    preds = clf.predict(test_set[features].values)
    test_preds3.append(preds)

In [ ]:
test_preds1 = np.stack(test_preds1).mean(0)
test_preds2 = np.stack(test_preds2).mean(0)
test_preds3 = np.stack(test_preds3).mean(0)

In [ ]:
test_preds = test_preds1*0.6 + test_preds2*0.3 + test_preds3*0.1

In [ ]:
submission = pd.DataFrame(data={'id': test_set.id, 'MedHouseVal': test_preds})
submission.head()

Rounding - the idea come from this [notebook](https://www.kaggle.com/code/dmitryuarov/ps-s3e1-coordinates-key-to-victory?scriptVersionId=115773179&cellId=36)

In [ ]:
vals = train_set['MedHouseVal'].unique().tolist()
submission['MedHouseVal'] = submission['MedHouseVal'].apply(lambda x: min(vals, key=lambda v: abs(v - x)))
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)